## Ngrams lab
LLM's and ChatGPT | Fall 2023 | McSweeney | CUNY Graduate Center

**Due:** September 17


### Background
The purpose of this lab is to explore ngram models. Ngram models are a good introduction to language models generally. Language models are probabilistic representations of language. Ngrams have the benefit of being easy to interrogate and relatively easy to understand (as compared to neural networks). 

In this lab, you will build an ngram model from the corpus of your choosing. The example is with 'The Great Gatsby' from Project Gutenberg, but there's a code block for any text file on your computer  


#### Notes
This lab is based heavily on the [nltk documentation](https://www.nltk.org/api/nltk.lm.html)

In [31]:
import numpy as np
import re

import nltk
# if you haven't downloaded punkt before, you only need to run the line below once 
#nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize

from nltk.util import bigrams
from nltk.lm.preprocessing import padded_everygram_pipeline

import requests

# Part 1
An example of how ngrams are generated

In [32]:
# you will need to leverage the requests package
r = requests.get(r'https://www.gutenberg.org/cache/epub/36/pg36.txt')
war_ot_worlds = r.text

# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t"]:
    war_ot_worlds = war_ot_worlds.replace(char, " ")

# check
print(war_ot_worlds[:100])

﻿The Project Gutenberg eBook of The War of the Worlds        This ebook is for the use of anyone any


In [33]:
# remove the metadata at the beginning - this is slightly different for each book
war_ot_worlds = war_ot_worlds[1928:]
print(war_ot_worlds[:150])

BOOK ONE  THE COMING OF THE MARTIANS          I.  THE EVE OF THE WAR.      No one would have believed in the last years of the nineteenth century  tha


#### Txt locally
If you'd rather use a file on your computer, here's the code -- you just need to save the text file in your local directory, and change the variables throughout. 

The example is a report from the [Congressional Research Service](https://www.everycrsreport.com/files/2020-11-10_R45178_62d6238caecf6c02ddf495be33b3439f09eed744.pdf) on AI and National Security.

In [34]:
# this is simplified for demonstration
def sample_clean_text(text: str):
    # lowercase
    text = text.lower()
    
    # remove punctuation from text
    text = re.sub(r"[^\w\s]", "", text)
    
    # tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # return your tokens
    return tokens

# call the function
sample_tokens = sample_clean_text(text = war_ot_worlds)

# check
print(sample_tokens[:50])

['book', 'one', 'the', 'coming', 'of', 'the', 'martians', 'i', 'the', 'eve', 'of', 'the', 'war', 'no', 'one', 'would', 'have', 'believed', 'in', 'the', 'last', 'years', 'of', 'the', 'nineteenth', 'century', 'that', 'this', 'world', 'was', 'being', 'watched', 'keenly', 'and', 'closely', 'by', 'intelligences', 'greater', 'than', 'mans', 'and', 'yet', 'as', 'mortal', 'as', 'his', 'own', 'that', 'as', 'men']


In [35]:
# create bigrams from the sample tokens
my_bigrams = bigrams(sample_tokens)

# check
list(my_bigrams)[:15]

[('book', 'one'),
 ('one', 'the'),
 ('the', 'coming'),
 ('coming', 'of'),
 ('of', 'the'),
 ('the', 'martians'),
 ('martians', 'i'),
 ('i', 'the'),
 ('the', 'eve'),
 ('eve', 'of'),
 ('of', 'the'),
 ('the', 'war'),
 ('war', 'no'),
 ('no', 'one'),
 ('one', 'would')]

# Part 2 - creating an ngram model


In [36]:
# 2 is for bigrams
n = 2
#specify the text you want to use
text = war_ot_worlds

Now we are going to use an NLTK shortcut for preprocessing. This will:
* pad all of the sentences with `<s>` and `</s>` to train on sentence boundaries, too.
* create both unigrams and bigrams
* create a training set and a full vocab to train on

We need to give it a pre-tokenized text (we'll use nltk's tokenizer)

In [37]:
# step 1: tokenize the text into sentences
sentences = nltk.sent_tokenize(text)

# step 2: tokenize each sentence into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# step 3: convert each word to lowercase
tokenized_text = [[word.lower() for word in sent] for sent in tokenized_sentences]

#notice the sentence breaks and what the first 10 items of the tokenized text
print(tokenized_text[0:4])

[['book', 'one', 'the', 'coming', 'of', 'the', 'martians', 'i', '.'], ['the', 'eve', 'of', 'the', 'war', '.'], ['no', 'one', 'would', 'have', 'believed', 'in', 'the', 'last', 'years', 'of', 'the', 'nineteenth', 'century', 'that', 'this', 'world', 'was', 'being', 'watched', 'keenly', 'and', 'closely', 'by', 'intelligences', 'greater', 'than', 'man', '’', 's', 'and', 'yet', 'as', 'mortal', 'as', 'his', 'own', ';', 'that', 'as', 'men', 'busied', 'themselves', 'about', 'their', 'various', 'concerns', 'they', 'were', 'scrutinised', 'and', 'studied', ',', 'perhaps', 'almost', 'as', 'narrowly', 'as', 'a', 'man', 'with', 'a', 'microscope', 'might', 'scrutinise', 'the', 'transient', 'creatures', 'that', 'swarm', 'and', 'multiply', 'in', 'a', 'drop', 'of', 'water', '.'], ['with', 'infinite', 'complacency', 'men', 'went', 'to', 'and', 'fro', 'over', 'this', 'globe', 'about', 'their', 'little', 'affairs', ',', 'serene', 'in', 'their', 'assurance', 'of', 'their', 'empire', 'over', 'matter', '.']]


In [38]:
print(tokenized_text[2])

['no', 'one', 'would', 'have', 'believed', 'in', 'the', 'last', 'years', 'of', 'the', 'nineteenth', 'century', 'that', 'this', 'world', 'was', 'being', 'watched', 'keenly', 'and', 'closely', 'by', 'intelligences', 'greater', 'than', 'man', '’', 's', 'and', 'yet', 'as', 'mortal', 'as', 'his', 'own', ';', 'that', 'as', 'men', 'busied', 'themselves', 'about', 'their', 'various', 'concerns', 'they', 'were', 'scrutinised', 'and', 'studied', ',', 'perhaps', 'almost', 'as', 'narrowly', 'as', 'a', 'man', 'with', 'a', 'microscope', 'might', 'scrutinise', 'the', 'transient', 'creatures', 'that', 'swarm', 'and', 'multiply', 'in', 'a', 'drop', 'of', 'water', '.']


Why tokenize sentences and words?

We want to be able to retain sentence boundaries to encode that, too.

In [39]:
# notice what the first 10 items of the vocabulary are:
#print(text[:10])
print(tokenized_text[0][:10])

['book', 'one', 'the', 'coming', 'of', 'the', 'martians', 'i', '.']


In [40]:
# we imported this function from nltk
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

In [41]:
from nltk.lm import MLE
# we imported this function from nltk linear models (lm) 
# it is for Maximum Likelihood Estimation

# MLE is the model we will use
lm = MLE(n)

In [42]:
# currently the vocab length is 0: it has no prior knowledge
len(lm.vocab)

0

In [43]:
# fit the model 
# training data is the bigrams and unigrams 
# the vocab is all the sentence tokens in the corpus 

lm.fit(train_data, padded_sents)
len(lm.vocab)

7567

In [44]:
help(MLE.fit) # this breaks the following code!  Not sure why?

Help on function fit in module nltk.lm.api:

fit(self, text, vocabulary_text=None)
    Trains the model on a text.
    
    :param text: Training text as a sequence of sentences.



In [45]:
#tokenized_text[0] 

In [46]:
# inspect the model's vocabulary. 
# be sure that a sentence you know exists (from tokenized_text) is in the 
print(lm.vocab.lookup(tokenized_text[3]))

('with', 'infinite', 'complacency', 'men', 'went', 'to', 'and', 'fro', 'over', 'this', 'globe', 'about', 'their', 'little', 'affairs', ',', 'serene', 'in', 'their', 'assurance', 'of', 'their', 'empire', 'over', 'matter', '.')


In [47]:
print(lm.vocab.lookup(["There", "is", "no", "spoon", "."]))

('<UNK>', 'is', 'no', 'spoon', '.')


In [48]:
my_list = ["There", "is", "no", "spoon", "."]
print(my_list)

['There', 'is', 'no', 'spoon', '.']


In [49]:
print(lm.vocab.lookup(my_list))

('<UNK>', 'is', 'no', 'spoon', '.')


In [50]:
print(lm.vocab.lookup(tokenized_text[0][0])) 

book


In [51]:
# see what happens when we include a word that is not in the vocab. 
print(lm.vocab.lookup('then wear the gold hat iphone .'.split()))

('then', 'wear', 'the', 'gold', 'hat', '<UNK>', '.')


What did the model replace 'iphone' with? 

Given that it didn't just return an "out of vocab" error, what does that mean about our model? 

Our model added '<UNK>' to our vocabulary so that when we train it on new data it will not crash when it encounters wordls that are not in our corpus.

In [52]:
# how many times does alien appear in the model?
print(lm.counts['alien'])

# what is the probability of alien appearing? 
# this is technically the relative frequency of alien appearing 
lm.score('alien')

1


1.2673628713373213e-05

In [53]:
# how many times does world appear in the model?
print(lm.counts['world'])

# what is the probability of world appearing? 
# this is technically the relative frequency of world appearing 
lm.score('world')

25


0.0003168407178343303

In [54]:
# how often does (world, and) occur and what is the relative frequency?
print(lm.counts[['world']]['war'])
lm.score('war', 'world'.split())

0


0.0

In [55]:
# how many times does martians appear in the model?
print(lm.counts['martians'])

# what is the probability of martians appearing? 
# this is technically the relative frequency of martians appearing 
lm.score('martians')

162


0.0020531278515664607

In [56]:
# how often does (martians, were) occur and what is the relative frequency?
print(lm.counts[['martians']]['were'])
lm.score('were', 'martians'.split())

16


0.09876543209876543

In [57]:
# From NLTK Documentation:
# Here’s how you get the score for a word given some preceding context. 

# For example we want to know what is the chance that “were” is preceded by “martians”.

lm.score("were", ["martians"])

0.09876543209876543

In [58]:
# what is the score of 'UNK'? 

lm.score("<UNK>")

0.0

Does the relative frequency of 'UNK' change your assumption about how the model behaves? 

How should we change our model to account for the fact the `<UNK>` words are not accounted for by the model? 

(We would want to implement Laplace smoothing or something similar to account for the unknown words per [NATURAL LANGUAGE PROCESSING N-gram language models: Part 1: Unigram model by Khanh Nguyen](https://medium.com/mti-technology/n-gram-language-model-b7c2fc322799)

Note: *Programmatically implementing this solution is beyond the scope of this course.*

## Generate text
We want to start our sentence with a word, and use that to predict all the words that come after that. We'll specify how long it should be. 

There is a certain amount of randomness encoded into n-gram models. This prevents a model from becoming entirely deterministic. Maximum Likelihood Estimation without some degree of randomness will only produce the most likely result every time. Setting Random Seed means we will get the same result every time. 

In [59]:
# generate a 20 word sentence starting with the word, 'martians'

print(lm.generate(20, text_seed= 'martians', random_seed=42))

['park', ',', 'and', 'down', 'the', 'railway', 'the', 'cart', '.', '</s>', 'air', '.', '</s>', 'a', 'persuasion', 'that', 'for', 'places', 'we', 'all']


This next code block is just to clean up the tokenized words and make them easier on human eyes. It is literally a detokenizer, which removes some extraneous text markup and reconciles some words back together. 

In [60]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(lm, num_words, text_seed, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in lm.generate(num_words, text_seed=text_seed, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [ ]:
# Now generate sentences that look much nicer. 
generate_sent(lm, 20, text_seed='martians', random_seed = 42)

Try a few more sentences, and try out another text. Once you are satisfied with what ngrams can (and cannot) do - post your code to your Github or another site. 